Lnagchain composes chains of components

ICEL and the runnable protocol define:

1. An allowed set of input types 

2. An allowed set of output types

3. Standard methods (invoke, stream, batch) - Means of modifying parameters at runtime (bind)

syntax looks like

chain = prompt | llm | OutputParser.

Interface :- 

1. Components implement Runnable protocol

2. Common methods includes:- 
    
    1. Invoke -synchronus method [ainvoke] -asynchronus
    
    2. stream -synchronus method [astream] -asynchronus

    3. batch -synchronus method [abatch] -asynchronus

3. Common properties:- 

    input_schema, output_schema

4. Common I/O

Component           |Input Type                                 |Output Type|
|:--:|:--:     |:--:|
Prompt              |Dictionary                                 |Prompt Value
|:--:|:--:     |:--:|
Retriever           |Single String                              |List of Documents
|:--:|:--:     |:--:|
LLM                 |String, List of messages or prompt value   |String
|:--:|:--:     |:--:|
ChatModel           |String, List of messages or prompt value   |ChatMessage
|:--:|:--:     |:--:|
Tool                |String/Dictionary                          |Tool Dependent
|:--:|:--:     |:--:|
OutputParser        |Output or LLM or ChatModel                 |Parser dependent
|:--:|:--:     |:--:|


Why use LCEL :- 

Runnables Support:-

1. Async, Batch and Streaming support

2. Fallbacks

3. Parallelism
    1. LLM calls can be time consuming
    2. Any components that can be run in parallel are!

4. Logging is built in

In [1]:
import os
from dotenv import load_dotenv

In [4]:
load_dotenv()
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

python-dotenv could not parse statement starting at line 1
python-dotenv could not parse statement starting at line 7


In [5]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

In [6]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)

In [7]:
model = ChatOpenAI()
output_parser = StrOutputParser()

/var/folders/s8/qyjb36g92fs3ztdqk120mmkw0000gn/T/ipykernel_5280/654874718.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI()


In [8]:
chain = prompt | model | output_parser

In [9]:
chain.invoke({"topic":"bears"})

'Why did the bear bring a flashlight to the party? \n\nBecause he wanted to be the life of the "bear"ty!'

In [10]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch


In [14]:
embeddings = OpenAIEmbeddings()
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding = embeddings
)

/var/folders/s8/qyjb36g92fs3ztdqk120mmkw0000gn/T/ipykernel_5280/2973660108.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [16]:
retriever = vectorstore.as_retriever()

In [18]:
retriever.get_relevant_documents("What does bears like to eat?")

[Document(metadata={}, page_content='bears like to eat honey'),
 Document(metadata={}, page_content='harrison worked at kensho')]

In [19]:
template = """Answer the question based only on following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

how to create this chain:- 

create a prompt that ask language model to answer the question based on following context.

variable -> Context and Question.

1. 1st the Only input to chain should be user question.

2. From user question we wanna fetch relevant context.

3. Pass that into Prompt

4. Pass that into Model

5. Pass that into Output parser and convert chat message into a string.

In [ ]:
# create something that takes in a single question and then turns it into dictionary of 2 elements context and questions

from langchain.schema.runnable import RunnableMap

In [22]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})| prompt | model | output_parser

In [23]:
chain.invoke({"question":"Where did Harrison work?"})

'Harrison worked at Kensho.'

to look what going underneath remove |

In [28]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

In [29]:
inputs.invoke({"question":"Where did Harrison work?"})

{'context': [Document(metadata={}, page_content='harrison worked at kensho'),
  Document(metadata={}, page_content='bears like to eat honey')],
 'question': 'Where did Harrison work?'}

One of the other things we can do with Runnables is we can bind parameters to them.

Bind and OpenAI Functions

In [31]:
functions = [
    {
        "name":"weather_search",
        "description":"Search for weather given an airport code",
        "parameters": {
            "type":"object",
            "properties":{
                "airport_code":{
                    "type":"string",
                    "description":"The airport code to get the weather for"
                },
            },
            "required":["airport_code"]
        }
    }
]

In [33]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("human","{input}")
    ]
)
model = ChatOpenAI(temperature=0).bind(functions=functions)

In [34]:
runnable = prompt | model

In [39]:
import json

In [42]:
runnable.invoke({"input":"What is the weather in sf"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'weather_search'}}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 64, 'total_tokens': 80, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run--3e42413d-d721-483f-bd6e-010c06fea0d1-0')

In [43]:
functions = [
    {
        "name":"weather_search",
        "description":"Search for weather given an airport code",
        "parameters": {
            "type":"object",
            "properties":{
                "airport_code":{
                    "type":"string",
                    "description":"The airport code to get the weather for"
                },
            },
            "required":["airport_code"]
        }
    },
    {
        "name":"sport_search",
        "description":"Search for news of recent sport event",
        "parameters": {
            "type":"object",
            "properties":{
                "team_name":{
                    "type":"string",
                    "description":"The sports team to search for"
                },
            },
            "required":["team_name"]
        }
    },
]

In [44]:
model = model.bind(functions=functions)

In [45]:
runnable = prompt | model

In [46]:
runnable.invoke({"input":"How did the patriots do yesterday"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"team_name":"patriots"}', 'name': 'sport_search'}}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 98, 'total_tokens': 116, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run--df1951a7-fb84-45f8-9dc8-f5f96aacf235-0')

Here model decided to use sport function as input given was related to sports.

Fallbacks

we are going to use older version of openai model which aren't as good as new ones

Chatmodels are newer type of models and they are generally good at things like outputing JSON, older models aren't.

In [47]:
from langchain.llms import OpenAI

In [48]:
simple_model = OpenAI(
    temperature = 0,
    max_tokens = 1000,
    model = 'text-davinci-001'
)

simple_chain = simple_model | json.loads

/var/folders/s8/qyjb36g92fs3ztdqk120mmkw0000gn/T/ipykernel_5280/1623018123.py:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  simple_model = OpenAI(


In [50]:
challenge = " Write three poems in a json blob, where each poem is a json of a title, author, and first line"

In [51]:
simple_model.invoke(challenge)

NotFoundError: Error code: 404 - {'error': {'message': 'The model `text-davinci-001` has been deprecated, learn more here: https://platform.openai.com/docs/deprecations', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

In [52]:
simple_chain.invoke(challenge)

NotFoundError: Error code: 404 - {'error': {'message': 'The model `text-davinci-001` has been deprecated, learn more here: https://platform.openai.com/docs/deprecations', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

In [54]:
model = ChatOpenAI(temperature=0)
chain = model | StrOutputParser() | json.loads

In [56]:
chain.invoke(challenge)

{'poem1': {'title': 'The Rose',
  'author': 'Emily Dickinson',
  'first_line': 'A rose by any other name would smell as sweet'},
 'poem2': {'title': 'The Road Not Taken',
  'author': 'Robert Frost',
  'first_line': 'Two roads diverged in a yellow wood'},
 'poem3': {'title': 'Hope is the Thing with Feathers',
  'author': 'Emily Dickinson',
  'first_line': 'Hope is the thing with feathers that perches in the soul'}}

In [57]:
final_chain = simple_chain.with_fallbacks([chain])

In [58]:
final_chain.invoke(challenge)

{'poem1': {'title': 'The Rose',
  'author': 'Emily Dickinson',
  'firstLine': 'A rose by any other name would smell as sweet'},
 'poem2': {'title': 'The Road Not Taken',
  'author': 'Robert Frost',
  'firstLine': 'Two roads diverged in a yellow wood'},
 'poem3': {'title': 'Hope is the Thing with Feathers',
  'author': 'Emily Dickinson',
  'firstLine': 'Hope is the thing with feathers that perches in the soul'}}

here using with_fallbacks, it 1st tried using simple_chain but output was not good or error occured so it changed to good chain

Interface

In [60]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = ChatOpenAI(temperature = 0)
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [61]:
chain.invoke({"topic":"bears"})

"Why did the bear break up with his girlfriend? \n\nBecause he couldn't bear the relationship anymore!"

In [63]:
chain.batch([{"topic":"bears"},{"topic":"frogs"}])

['Why did the bear bring a flashlight to the party? \n\nBecause he heard it was going to be a "beary" good time!',
 'Why are frogs so happy? Because they eat whatever bugs them!']

In [64]:
for t in chain.stream({"topic":"bears"}):
    print(t)


Why
 did
 the
 bear
 bring
 a
 flashlight
 to
 the
 party
?
 


Because
 he
 heard
 it
 was
 going
 to
 be
 a
 "
be
ary
"
 good
 time
!



asynchronous methods, all methods have same asynchrnous methods

In [66]:
response = await chain.ainvoke({"topic":"bears"})
response

'Why did the bear bring a flashlight to the party? \n\nBecause he heard it was going to be a "beary" good time!'